In [2]:
import numpy as np
from scipy import linalg as lg

# TT-SVD

In [25]:
def use_rank(s, eps):
    k = 1
    spec = np.sum(s**2)
    while k < s.shape[0] and np.sum(s[k:]**2) >= eps:
        k = k + 1
    return k, np.sum(s[k:]**2)

In [60]:
def TT_SVD(T, eps=10**(-8), ranks=None):
    eps = (eps * np.linalg.norm(T.reshape(-1,)))**2

    factors = []
    dims = list(T.shape)
    d = len(dims)

    if ranks and len(ranks) != d - 1:
        print(f"Ranks must have {d - 1} elements")
        return 0

    T = T.reshape(dims[0], -1)

    for i in range(d - 1):

        U, s, Vh = np.linalg.svd(T, full_matrices=False)

        if ranks:
            rank = ranks[i]
        else:
            rank, m_eps = use_rank(s, eps / (d - i))
            eps = eps - m_eps

        T = s[:rank].reshape((-1, 1)) * Vh[:rank, :]

        if i == 0:
            factors.append(U[:, :rank])
            T = T.reshape(rank * dims[i + 1], -1)
        elif i != d - 2:
            factors.append(U[:, :rank].reshape(dims[i - 1], dims[i], rank))
            T = T.reshape(rank * dims[i + 1], -1)
        else:
            factors.append(U[:, :rank].reshape(dims[i - 1], dims[i], rank))
            factors.append(T)

        dims[i] = rank

    return factors, dims[:-1]

# TT-orthogonalization

In [204]:
def TT_orthogonalization(factors, ranks):
    d = len(factors)
    new_ranks = []

    for i in range(d):

        if not i:
            factors[i], R = np.linalg.qr(factors[i])
            new_ranks.append(R.shape[0])
        elif i != d - 1:
            timeless_factor = R @ factors[i].reshape(ranks[i - 1], -1)
            timeless_factor = timeless_factor.reshape(ranks[i - 1] * factors[i].shape[1], -1)
            timeless_factor, R = np.linalg.qr(timeless_factor)
            new_ranks.append(R.shape[0])
            factors[i] = timeless_factor.reshape(new_ranks[-2], factors[i].shape[1], new_ranks[-1])
        else:
            factors[i] = R @ factors[i]

    return factors, new_ranks

# TT-compression

In [457]:
def TT_compression(factors, ranks, eps=10**(-8), new_ranks=None, is_ort=False):
    if not is_ort:
        factors, ranks = TT_orthogonalization(factors, ranks)

    norm_tensor = np.linalg.norm(factors[-1].reshape(-1,))

    eps = (eps * norm_tensor)**2
    d = len(factors)

    for i in range(d - 1, -1, -1):
        if i == d - 1:
            U, s, Vh = np.linalg.svd(factors[i], full_matrices=False)

            if new_ranks:
                rank = new_ranks[i - 1]
            else:
                rank, m_eps = use_rank(s, eps / (d - i))
                eps = eps - m_eps

            factors[i] = Vh[:rank, :]
            Z = U[:, :rank] * s[:rank]

            ranks[i - 1] = rank

        elif i != 0:
            timeless_factor = factors[i].reshape(ranks[i - 1] * factors[i].shape[1], -1) @ Z
            U, s, Vh = np.linalg.svd(timeless_factor.reshape(ranks[i - 1], -1), full_matrices=False)

            if new_ranks:
                rank = new_ranks[i - 1]
            else:
                rank, m_eps = use_rank(s, eps / (d - i))
                eps = eps - m_eps

            factors[i] = Vh[:rank, :].reshape(rank, factors[i].shape[1], ranks[i])

            Z = U[:, :rank] * s[:rank]
            ranks[i - 1] = rank
        else:
            factors[i] = factors[i] @ Z


    return factors, ranks

# Tensor from TT

In [199]:
def TT_to_tensor(factors, ranks):
    d = len(factors)

    T = factors[0]
    dims = [factors[0].shape[0]]

    for i in range(1, d):
        if i != d - 1:
            dims.append(factors[i].shape[1])
            T = T @ factors[i].reshape(ranks[i - 1], -1)
            T = T.reshape(-1, ranks[i])
        else:
            dims.append(factors[i].shape[1])
            T = T @ factors[i].reshape(ranks[i - 1], -1)
            T = T.reshape(dims)

    return T, dims

# Create Tensor

In [506]:
sizes = np.array((10, 20, 30, 40))
T = np.zeros(sizes)
for i in range(sizes[0]):
    for j in range(sizes[1]):
        for k in range(sizes[2]):
            for m in range(sizes[3]):
                T[i, j, k, m] = 1 / (i + j + k + m + 1)

# Test TT-SVD

In [210]:
factors, ranks = TT_SVD(T.copy(), eps=10**(-10))

In [211]:
ranks

[10, 12, 12]

In [152]:
new_T, dims = TT_to_tensor(factors.copy(), ranks.copy())

In [153]:
print(np.linalg.norm((new_T - T).reshape(-1,)) / np.linalg.norm((T).reshape(-1,)))

2.720835568093407e-11


# Test TT_orthogonalization

In [496]:
factors, ranks = TT_SVD(T.copy(), eps=10**(-10))

In [497]:
ranks

[10, 12, 12]

In [498]:
for i in range(len(factors)):
    factors[i] += np.ones(factors[i].shape)

Факторы не ортогональны

In [499]:
print(np.linalg.norm((factors[0].T @ factors[0] - np.eye(factors[0].shape[1])).reshape(-1,)) / np.linalg.norm((np.eye(factors[0].shape[1])).reshape(-1,)))
for i in range(1, len(factors) - 1):
    a, b, c = factors[i].shape
    new_fact = factors[i].reshape(a * b, c)
    print(np.linalg.norm((new_fact.T @ new_fact - np.eye(c)).reshape(-1,)) / np.linalg.norm((np.eye(c)).reshape(-1,)))

30.26232614678093
689.6290871268446
1248.6179141523971


In [500]:
new_T, dims = TT_to_tensor(factors.copy(), ranks.copy())

In [501]:
ort_factors, ort_ranks = TT_orthogonalization(factors, ranks)

In [502]:
ort_ranks

[10, 12, 12]

Факторы ортогональны

In [503]:
print(np.linalg.norm((ort_factors[0].T @ ort_factors[0] - np.eye(ort_factors[0].shape[1])).reshape(-1,)) / np.linalg.norm((np.eye(ort_factors[0].shape[1])).reshape(-1,)))
for i in range(1, len(ort_factors) - 1):
    a, b, c = ort_factors[i].shape
    new_fact = ort_factors[i].reshape(a * b, c)
    print(np.linalg.norm((new_fact.T @ new_fact - np.eye(c)).reshape(-1,)) / np.linalg.norm((np.eye(c)).reshape(-1,)))

3.728395895188642e-16
6.480096795978864e-16
5.674915164924475e-16


In [504]:
ort_T, dims = TT_to_tensor(ort_factors.copy(), ort_ranks.copy())

Абсолютная ошибка между построенным тензором и исходным

In [505]:
print(np.linalg.norm((new_T - ort_T).reshape(-1,)) / np.linalg.norm((new_T).reshape(-1,)))

5.26250155948656e-16


# Test TT-compress

Та же точность

In [379]:
factors, ranks = TT_SVD(T.copy(), eps=10**(-10))

In [380]:
ranks

[10, 12, 12]

In [381]:
new_T, dims = TT_to_tensor(factors.copy(), ranks.copy())

In [382]:
print(np.linalg.norm((new_T - T).reshape(-1,)) / np.linalg.norm((T).reshape(-1,)))

2.720835568093407e-11


In [383]:
comp_factors, comp_ranks = TT_compression(factors.copy(), ranks.copy(), eps=10**(-10), is_ort=True)

In [384]:
comp_ranks

[9, 12, 12]

In [385]:
comp_T, comp_dims = TT_to_tensor(comp_factors.copy(), comp_ranks.copy())

In [386]:
print(np.linalg.norm((comp_T - T).reshape(-1,)) / np.linalg.norm((T).reshape(-1,)))

6.226936244242895e-11


Другая точность

In [447]:
factors, ranks = TT_SVD(T.copy(), eps=10**(-10))
print(ranks)
new_T, dims = TT_to_tensor(factors.copy(), ranks.copy())
print(np.linalg.norm((new_T - T).reshape(-1,)) / np.linalg.norm((T).reshape(-1,)))

[10, 12, 12]
2.720835568093407e-11


In [448]:
comp_factors, comp_ranks = TT_compression(factors.copy(), ranks.copy(), eps=10**(-3), is_ort=True)
comp_ranks
comp_T, comp_dims = TT_to_tensor(comp_factors.copy(), comp_ranks.copy())
print(np.linalg.norm((comp_T - T).reshape(-1,)) / np.linalg.norm((T).reshape(-1,)))

0.00035992451566143355


С ортогонализацией

In [519]:
factors, ranks = TT_SVD(T.copy(), eps=10**(-10))
print(ranks)
for i in range(len(factors)):
    factors[i] += np.ones(factors[i].shape)
new_T, dims = TT_to_tensor(factors.copy(), ranks.copy())

[10, 12, 12]


In [520]:
print(np.linalg.norm((factors[0].T @ factors[0] - np.eye(factors[0].shape[0])).reshape(-1,)) / np.linalg.norm((np.eye(factors[0].shape[0])).reshape(-1,)))
for i in range(1, len(factors) - 1):
    a, b, c = factors[i].shape
    new_fact = factors[i].reshape(a * b, c)
    print(np.linalg.norm((new_fact.T @ new_fact - np.eye(c)).reshape(-1,)) / np.linalg.norm((np.eye(c)).reshape(-1,)))

30.26232614678093
689.6290871268446
1248.6179141523971


In [521]:
comp_factors, comp_ranks = TT_compression(factors, ranks, eps=10**(-5), is_ort=False)
print(comp_ranks)
comp_T, comp_dims = TT_to_tensor(comp_factors.copy(), comp_ranks.copy())
print(np.linalg.norm((comp_T - new_T).reshape(-1,)) / np.linalg.norm((new_T).reshape(-1,)))

[9, 7, 4]
6.077925068510356e-06


In [522]:
for i in range(1, len(comp_factors) - 1):
    a, b, c = comp_factors[i].shape
    new_fact = comp_factors[i].reshape(a, b * c)
    print(np.linalg.norm((new_fact @ new_fact.T - np.eye(a)).reshape(-1,)) / np.linalg.norm((np.eye(a)).reshape(-1,)))


print(np.linalg.norm((comp_factors[-1] @ comp_factors[-1].T - np.eye(comp_factors[-1].shape[0])).reshape(-1,)) / np.linalg.norm((np.eye(comp_factors[-1].shape[1])).reshape(-1,)))

1.0721250953730498e-15
8.342077254213625e-16
2.2051753995872503e-16
